# The PICO model

The model is based on Reese et al (2018): "Antarctic sub-shelf melt rates via PICO"

In part (a) we test a few idealized geometries, in part (b) realistic geometries are presented.

There are a few differences to the original implementation w.r.t to real geometries.
- underlying datasets: we use the BedMachine2 data
- model resolution: we use the BedMachine native grid at 500 m grid spacing, whereas PICO uses 5 km

In [ ]:
import sys
import numpy as np
import xarray as xr
import pandas as pd
import warnings
import geopandas
import matplotlib
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

sys.path.append("..")
# matplotlib.rc_file('../rc_file')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore", category=matplotlib.MatplotlibDeprecationWarning)

from real_geometry import RealGeometry, glaciers
from PICO import PicoModel, table2
from compare_models import compare_PICO

## (a) idealized geometries

In [ ]:
f, ax = plt.subplots(5,3, figsize=(12,12), sharey='row', constrained_layout=True)
for i, testcase in enumerate(['test1', 'test2', 'test3']):
    geo, ds = PicoModel(name=testcase).compute_pico()
    geo.draft.plot(ax=ax[0,i])
    ax[0,i].set_title(testcase)
    ds.melt.plot(ax=ax[1,i])
    ds.mk.plot(ax=ax[2,i])
    ds.Tk.plot(ax=ax[3,i])
    ds.Sk.plot(ax=ax[4,i])

These are test geometries, the `test1` is a quasi-1D iceshelf of 100 km length with a grounding line depth of 1000 m and an ice shelf front depth of 500 m. `test2` is simply a rotated version of `test1`. `test3` has a sinusoidal grounding line profile and a flat ice shelf front profile. The geometries (arbitrarily) have 3 boxes. `boxnr=0` represents either the average (for melt) or the ambient conditions (temperature and salinity).

The melt is highest near the grounding line in part because in-situ temperatures are highest there. Both temperature and salinity decrease as the plume ascends towards the ice shelf front.

## (b) real geometries
At first execution, the code creates the real geometries from the BedMachine data and IceVelocity data (these files are too big for version control on Github, but see lines 26f in `real_geometries.py` for their location).

### example: Thwaites glacier

In [ ]:
geo, ds = PicoModel('Thwaites').compute_pico()

In [ ]:
f, ax = plt.subplots(1,4, figsize=(20,4), sharey=True)
geo.draft.plot(ax=ax[0])
geo.rd.plot(ax=ax[1])
geo.box.plot(ax=ax[2])
ds.melt.plot(ax=ax[3])


### comparing the 6 currently implemented ice shelves

In [ ]:
for i, glacier in enumerate(glaciers):
    if glacier in ['Ross', 'FilchnerRonne']:  # at the BedMachine resolution, these datasets are too big for laptop memory
        continue
    PicoModel(glacier).compute_pico()

In [ ]:
compare_PICO()

### maps of Amundsen Sea and East Antarctica

In [ ]:
proj = ccrs.SouthPolarStereo(true_scale_latitude=-71)
def fn_poly(glacier):  return f'../../data/mask_polygons/{glacier}_polygon.geojson'
x5, y5, _, _ = geopandas.read_file(fn_poly('MoscowUniversity'), crs='espg:3031').total_bounds
_, _, x6, y6 = geopandas.read_file(fn_poly('Totten'), crs='espg:3031').total_bounds
x3, _, _, y4 = geopandas.read_file(fn_poly('PineIsland'), crs='espg:3031').total_bounds
_, y3, x4, _ = geopandas.read_file(fn_poly('Dotson'), crs='espg:3031').total_bounds

import matplotlib.ticker as mticker

f = plt.figure(figsize=(8,12))
for i in range(2):  # Amundsen Sea, Totten+MoscowUniversity
    (x1,x2,y1,y2) = [(x3,x4,y3-1e4,y4+2e4),(x5-1e4,x6,y5,y6+1e4)][i]
    shelves = [['PineIsland','Thwaites','Dotson'], ['Totten','MoscowUniversity']][i]
    for s, shelf in enumerate(shelves):
        (x,y) = [[(.65,.88),(.05,.55),(.05,.2)],[(.3,.8),(.4,.1)]][i][s]
        name = [['Pine\nIsland','Thwaites','Dotson/\nCrosson'], ['Totten','Moscow\nUniversity']][i][s]
        dsg = xr.open_dataset(RealGeometry(shelf).fn_PICO)
        dsP = xr.open_dataset(PicoModel(shelf).fn_PICO_output)
        lon, lat = dsg.lon, dsg.lat
        for j in range(3):
            q = [dsg.draft, dsg.box.where(dsg.mask), dsP.melt.where(dsg.mask)][j]
            cmap = ['viridis', 'Spectral','inferno_r'][j]
            (vmin,vmax) = [(-2000,0),(1,2),(0,25)][j]
            ax = f.add_axes([j/3,.545-.54*i,.33,.45], projection=proj)
            ax.set_frame_on(False)
            ax.set_extent([x1,x2,y1,y2], crs=proj)
            ax.coastlines()
            gl = ax.gridlines()
            
            gl.xlocator = mticker.FixedLocator(np.arange(-180,179,5))
            gl.ylocator = mticker.FixedLocator(np.arange(-89,89))
            
            im = ax.pcolormesh(lon, lat, q, transform=ccrs.PlateCarree(),
                          cmap=cmap, vmin=vmin, vmax=vmax)
            if i==0:  # colorbars
                cax = f.add_axes([j/3+.02,.5,.29,.02])
                label = ['draft [m]', 'box nr.', 'melt rate [m/yr]'][j]
                plt.colorbar(im, cax=cax, orientation='horizontal', label=label)
            if j==0:  ax.text(x, y, name, weight='bold', transform=ax.transAxes)
            if j==2:  ax.text(x, y, f'{dsP.mk[0].values:.2f} m/yr', transform=ax.transAxes)

In [ ]:
# f, ax = plt.subplots(4, 3, figsize=(15,15))
# for i, key in enumerate(list(ds.keys())[:-1]):
#     if i<9:  kwargs = {'cbar_kwargs':{'orientation':'horizontal'}}
#     else:    kwargs = {}
#     ds[key].plot(ax=ax[int(i/3), i%3], **kwargs)